In [131]:
import pandas as pd
from requests_html import HTMLSession
import geocoder
import folium

In [132]:
session = HTMLSession()
r = session.get("https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=03%2F01%2F2024&endDate=03%2F10%2F2024")
num_pages = int(r.html.search("1 / {}")[0])

In [133]:
df = pd.DataFrame()
for i in range(num_pages):
    link = f"https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=03%2F01%2F2024&endDate=03%2F10%2F2024&offset={i*5}"
    table = pd.read_html(link)[0]
    df = pd.concat([df, table], ignore_index = True)

In [134]:
df["Location"] = df["Location"].str.replace(" \\(.*", "", regex = True)
df = df[df.Incident.str.match('Theft|Robbery|Battery|Assault')]
df["Location"] = df["Location"] + ", Chicago, IL"
df["Location"] = df["Location"].apply(lambda add: geocoder.osm(add).latlng)
crime_table = df.filter(items = ["Incident", "Location", "Reported"])

In [135]:
min_lng, max_lng = -87.610, -87.583
min_lat, max_lat = 41.783, 41.805

m = folium.Map(
    location = [41.793983, -87.594633],
    tiles = "Cartodb Positron",
    zoom_start = 14,
    min_zoom = 14,
    max_bounds = True,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lng,
    max_lon=max_lng,
    )

In [136]:
crime_table = crime_table.replace(r'.*Theft.*', 'Theft', regex = True)
crime_table = crime_table.replace(r'.*Robbery.*', 'Armed Robbery', regex = True)
crime_table = crime_table.replace(r'.*Battery|Assault.*', 'Battery/Assault', regex = True)

In [137]:
colors = {
    "Theft" : "pink",
    "Battery/Assault" : "orange",
    "Armed Robbert" : "red" 
    }

crime_table.apply(lambda row:folium.CircleMarker(location=[row["Location"][0], 
                                                           row["Location"][1]], 
                                                           radius=5,
                                                           stroke = False,
                                                           fill_color = colors[row["Incident"]],
                                                           fill_opacity = 0.85,
                                                           popup = row["Incident"])
                                                           .add_to(m), axis=1)


2     <folium.vector_layers.CircleMarker object at 0...
6     <folium.vector_layers.CircleMarker object at 0...
8     <folium.vector_layers.CircleMarker object at 0...
12    <folium.vector_layers.CircleMarker object at 0...
13    <folium.vector_layers.CircleMarker object at 0...
20    <folium.vector_layers.CircleMarker object at 0...
21    <folium.vector_layers.CircleMarker object at 0...
dtype: object

In [138]:
m